## Install libraries

In [10]:
import os
import textract
from spacy.pipeline import EntityRuler
from spacy import displacy
import jsonlines
from spacy.lang.en import English
from spacy.tokens import Doc
import spacy
import PyPDF2
import json, requests

## 1. Upload resume

In [1]:
!pip install PyPDF2

In [2]:
import PyPDF2

file = '/content/Pankov_D_CV.pdf'

def extract_text_from_pdf(file):
   
    fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
    page_count = fileReader.getNumPages()
    text = [fileReader.getPage(i).extractText() for i in range(page_count)]
    
    return str(text).replace("\\n", "")

def extract_candidate_skills(file):
    candidate_skills = get_skills(extract_text_from_pdf(file))  
        
    return candidate_skills

cand_skills = extract_candidate_skills(file)

In [11]:
# !pip install -U ipywidgets

In [10]:
# !sudo jupyter nbextension install --py widgetsnbextension

In [13]:
# from ipywidgets import FileUpload
# from IPython.display import display
# upload = FileUpload(accept='.txt', multiple=True)
# display(upload)

FileUpload(value={}, accept='.txt', description='Upload', multiple=True)

In [ ]:
from spacy.lang.en import English

nlp = English()

# File Extension. set as 'pdf' or as 'doc(x)'
extension = 'pdf'

def create_tokenized_texts_list(extension):
    '''Create two lists, one with the names of the candidate and one with the tokenized 
       resume texts extracted from either a .pdf or .doc'''
    resume_texts, resume_names = [], []
    
    # Loop over the contents of the directory containing the resumes, filtering by .pdf or .doc(x)
    for resume in list(filter(lambda x: extension in x, os.listdir(PROJECT_DIR + '/CV'))):
        if extension == 'pdf':
            # Read in every resume with pdf extension in the directory
            resume_texts.append(nlp(extract_text_from_pdf(PROJECT_DIR + '/CV/' + resume)))
        elif 'doc' in extension:
            # Read in every resume with .doc or .docx extension in the directory
            resume_texts.append(nlp(extract_text_from_word(PROJECT_DIR + '/CV/' + resume)))
            
        resume_names.append(resume.split('_')[0].capitalize())
        
    return resume_texts, resume_names

In [11]:
nlp = spacy.load('en_core_web_sm')

skill_pattern_path = "jz_skill_patterns.jsonl"
with jsonlines.open("jz_skill_patterns.jsonl") as f:
    created_entities = [line['label'].upper() for line in f.iter()]

ruler = EntityRuler(nlp).from_disk(skill_pattern_path)

nlp.add_pipe(ruler, after='parser')

def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_=="SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def extract_text_from_pdf(file):
    fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
    page_count = fileReader.getNumPages()
    text = [fileReader.getPage(i).extractText() for i in range(page_count)]
    
    return str(text).replace("\\n", "")

In [12]:
file = 'CV.pdf'

In [14]:
cv_skills = get_skills(extract_text_from_pdf(file))
cv_skills

['C',
 'support',
 'analytics',
 'Marketing',
 'Google',
 'monitoring',
 'Play',
 'Marketing',
 'analytics',
 'Data analysis',
 'Marketing',
 'Marketing',
 'LANGUAGES',
 'SOFTWARE',
 'analytics',
 'Jupyter',
 'Superset',
 'Marketing',
 'Google',
 'Business',
 'Marketing',
 'Google Analytics',
 'Google Tag Manager',
 'Analytics',
 'Design',
 'Python',
 'Pandas',
 'Numpy',
 'Java',
 'SQL',
 'HTML',
 'CSS',
 'Git',
 'Linux',
 'Marketing',
 'Google Analytics']

## 2. Search for a job

### Reddit API

In [19]:
# ! pip install praw timeago

In [25]:
import datetime
import praw
import timeago


class Job():
    def __init__(self,post):
        self.title = post.title
        self.date = compute_ago(post.created_utc)
        self.url = post.url


def compute_ago(created):
    date = datetime.datetime.fromtimestamp(int(float(str(created))))
    now = datetime.datetime.now()
    print (timeago.format(date, now))
    return timeago.format(date, now)


client = "xxxx" # Client ID
secret = "xxxxxxxxxx" # Secret key

# Function to connect to Reddit API
def connect(cl_id, cl_secret):
    return praw.Reddit(client_id = cl_id, client_secret = cl_secret, user_agent = "Jobzilla")

def search_reddit(query):
    red = connect(client, secret)
    # Generating the string for keywords
    keys = query.split(",")
    subs = ["jobbit", "forhire"]

    # Generate string for keys
    str_keys = "(title : \"" +keys[0]+ "\""
    for k in keys[1:]:
        str_keys = str_keys + " OR title:\"" + k + "\""
    str_keys = str_keys + ")"

    # Generate string for subs
    str_subs = "(subreddit:hiring"
    for sub in subs:
        str_subs = str_subs + " OR subreddit:" + sub
    str_subs = str_subs + ") "

    # Creating the full search query using Reddit search syntax
    full_str = "(title:\"hiring\" OR flair:Hiring) AND  " + str_keys + " AND " + str_subs

    # making the call
    all = red.subreddit("all")
    """
    for post in all.search(query = full_str, sort = "new"):
        print post.title
        post_date = datetime.datetime.utcfromtimestamp(post.created_utc)
        print str(post_date.month) + ", " + str(post_date.day) + " in " + str(post_date.year)
    """
    return all.search(query = full_str, sort = "new")
    # return all

In [30]:
keyword = 'python, sql'

all = search_reddit(query = keyword)

In [1]:
dict = { "title":[],
                "subreddit":[],
                "score":[], 
                "id":[], 
                "url":[], 
                "comms_num": [], 
                "created": [], 
                "body":[]}

for subreddit in all:
    for submission in reddit.subreddit(subreddit):               
        dict["title"].append(submission.title)
        dict['subreddit'].append(submission.subreddit)
        dict["score"].append(submission.score)
        dict["id"].append(submission.id)
        dict["url"].append(submission.url)
        dict["comms_num"].append(submission.num_comments)
        dict["created"].append(submission.created)
        dict["body"].append(submission.selftext)

In [ ]:
df = pd.DataFrame(dict)
df.head()

In [ ]:
df.to_csv('jobs_reddit_2.csv')

## 3. Create cover letter

In [14]:
prefixtext="<|startoftext|>~['Analytics','business','data science', 'business', 'algorithms']~Freelancer - Marketing Analyst"

In [15]:
import requests
req = requests.post('https://jzl-api-v7otpcjevq-lz.a.run.app',
                    json={'length': 500, 'temperature': 1.0, 'prefix':prefixtext})

In [ ]:
text = req.json()['text']
print(text)